<a href="https://colab.research.google.com/github/Gwan98/Project/blob/main/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5;pip install -qr requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!mkdir /content/gdrive/MyDrive/data

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pwd

/content/gdrive/MyDrive


In [ ]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
from typing_extensions import IntVar
import json
import os
import random


def split_dataset(input_json, output_dir, val_ratio, test_ratio, random_seed):
    random.seed(random_seed)

    with open(input_json) as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images']
    annotations = dataset['annotations']
    categories = dataset['categories']

    # file_name에 prefix 디렉토리까지 포함 (CocoDataset 클래스를 사용하는 경우)
    #for image in images:
        #image['file_name'] = '{}/{}'.format(image['file_name'][0], image['file_name'])

    image_ids = [x.get('id') for x in images]
    image_ids.sort()
    random.shuffle(image_ids)

    num_val = int(len(image_ids) * val_ratio)
    num_test = int(len(image_ids) * test_ratio)
    num_train = int(len(image_ids)) - num_val - num_test

    image_ids_val, image_ids_train , image_ids_test= set(image_ids[:num_val]), set(image_ids[num_val:num_train]), set(image_ids[num_train:])

    train_images = [x for x in images if x.get('id') in image_ids_train]
    val_images = [x for x in images if x.get('id') in image_ids_val]
    test_images = [x for x in images if x.get('id') in image_ids_test]
    train_annotations = [x for x in annotations if x.get('image_id') in image_ids_train]
    val_annotations = [x for x in annotations if x.get('image_id') in image_ids_val]
    test_annotations = [x for x in annotations if x.get('image_id') in image_ids_test]

    train_data = {
        'images': train_images,
        'annotations': train_annotations,
        'categories': categories,
    }

    val_data = {
        'images': val_images,
        'annotations': val_annotations,
        'categories': categories,
    }

    test_data = {
        'images': test_images,
        'annotations': test_annotations,
        'categories': categories,
    }

    output_seed_dir = os.path.join(output_dir, f'seed{random_seed}')
    os.makedirs(output_seed_dir, exist_ok=True)
    output_train_json = os.path.join(output_seed_dir, 'train.json')
    output_val_json = os.path.join(output_seed_dir, 'val.json')
    output_test_json = os.path.join(output_seed_dir, 'test.json')
    output_train_csv = os.path.join(output_seed_dir, 'train.csv')
    output_val_csv = os.path.join(output_seed_dir, 'val.csv')
    output_test_csv = os.path.join(output_seed_dir, 'test.csv')

    print(f'write {output_train_json}')
    with open(output_train_json, 'w') as train_writer:
        json.dump(train_data, train_writer)

    print(f'write {output_val_json}')
    with open(output_val_json, 'w') as val_writer:
        json.dump(val_data, val_writer)

    print(f'write {output_test_json}')
    with open(output_test_json, 'w') as test_writer:
        json.dump(test_data, test_writer)

In [ ]:
split_dataset(input_json='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/annotation.json',
              output_dir='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json',
              val_ratio=0.2,
              test_ratio=0.1,
              random_seed=221112)

//실행완료

write /content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/train.json
write /content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/val.json
write /content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/test.json


In [ ]:
!git clone https://github.com/alexmihalyk23/COCO2YOLO.git
//실행완료

Cloning into 'COCO2YOLO'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 63 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [ ]:
import shutil

class COCO2YOLO:
  # 소스 이미지 디렉토리와 Json annotation 파일, 타겟 이미지 디렉토리, 타겟 annotation 디렉토리를 생성자로 입력 받음. 
  def __init__(self, src_img_dir, json_file, tgt_img_dir, tgt_anno_dir):
    self.json_file = json_file
    self.src_img_dir = src_img_dir
    self.tgt_img_dir = tgt_img_dir
    self.tgt_anno_dir = tgt_anno_dir
    # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
    self._check_file_and_dir(json_file, tgt_img_dir, tgt_anno_dir)
    # json 파일을 메모리로 로딩. 
    self.labels = json.load(open(json_file, 'r', encoding='utf-8'))
    # category id와 이름을 매핑하지만, 실제 class id는 이를 적용하지 않고 별도 적용. 
    self.coco_id_name_map = self._categories()
    self.coco_name_list = list(self.coco_id_name_map.values())
    print("total images", len(self.labels['images']))
    print("total categories", len(self.labels['categories']))
    print("total labels", len(self.labels['annotations']))
  
  # json 파일과 타겟 디렉토리가 존재하는지 확인하고, 디렉토리의 경우는 없으면 생성. 
  def _check_file_and_dir(self, file_path, tgt_img_dir, tgt_anno_dir):
    if not os.path.exists(file_path):
        raise ValueError("file not found")
    if not os.path.exists(tgt_img_dir):
        os.makedirs(tgt_img_dir)
    if not os.path.exists(tgt_anno_dir):
        os.makedirs(tgt_anno_dir)

  # category id와 이름을 매핑하지만, 추후에 class 명만 활용. 
  def _categories(self):
    categories = {}
    for cls in self.labels['categories']:
        categories[cls['id']] = cls['name']
    return categories
  
  # annotation에서 모든 image의 파일명(절대 경로 아님)과 width, height 정보 저장. 
  def _load_images_info(self):
    images_info = {}
    for image in self.labels['images']:
        id = image['id']
        file_name = image['file_name']
        if file_name.find('\\') > -1:
            file_name = file_name[file_name.index('\\')+1:]
        w = image['width']
        h = image['height']
  
        images_info[id] = (file_name, w, h)

    return images_info

  # ms-coco의 bbox annotation은 yolo format으로 변환. 좌상단 x, y좌표, width, height 기반을 정규화된 center x,y 와 width, height로 변환. 
  def _bbox_2_yolo(self, bbox, img_w, img_h):
    # ms-coco는 좌상단 x, y좌표, width, height
    x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
    # center x좌표는 좌상단 x좌표에서 width의 절반을 더함. center y좌표는 좌상단 y좌표에서 height의 절반을 더함.  
    centerx = bbox[0] + w / 2
    centery = bbox[1] + h / 2
    # centerx, centery, width, height를 이미지의 width/height로 정규화. 
    dw = 1 / img_w
    dh = 1 / img_h
    centerx *= dw
    w *= dw
    centery *= dh
    h *= dh
    return centerx, centery, w, h
  
  # image와 annotation 정보를 기반으로 image명과 yolo annotation 정보 가공. 
  # 개별 image당 하나의 annotation 정보를 가지도록 변환. 
  def _convert_anno(self, images_info):
    anno_dict = dict()
    for anno in self.labels['annotations']:
      bbox = anno['bbox']
      image_id = anno['image_id']
      category_id = anno['category_id']

      image_info = images_info.get(image_id)
      image_name = image_info[0]
      img_w = image_info[1]
      img_h = image_info[2]
      yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

      anno_info = (image_name, category_id, yolo_box)
      anno_infos = anno_dict.get(image_id)
      if not anno_infos:
        anno_dict[image_id] = [anno_info]
      else:
        anno_infos.append(anno_info)
        anno_dict[image_id] = anno_infos
    return anno_dict

  # class 명을 파일로 저장하는 로직. 사용하지 않음. 
  def save_classes(self):
    sorted_classes = list(map(lambda x: x['name'], sorted(self.labels['categories'], key=lambda x: x['id'])))
    print('coco names', sorted_classes)
    with open('coco.names', 'w', encoding='utf-8') as f:
      for cls in sorted_classes:
          f.write(cls + '\n')
    f.close()
  # _convert_anno(images_info)로 만들어진 anno 정보를 개별 yolo anno txt 파일로 생성하는 로직. 
  # coco2yolo()에서 anno_dict = self._convert_anno(images_info)로 만들어진 anno_dict를 _save_txt()에 입력하여 파일 생성
  def _save_txt(self, anno_dict):
    # 개별 image별로 소스 image는 타겟이미지 디렉토리로 복사하고, 개별 annotation을 타겟 anno 디렉토리로 생성. 
    for k, v in anno_dict.items():
      # 소스와 타겟 파일의 절대 경로 생성. 
      src_img_filename = os.path.join(self.src_img_dir, v[0][0])
      tgt_anno_filename = os.path.join(self.tgt_anno_dir,v[0][0].split(".")[0] + ".txt")
      #print('source image filename:', src_img_filename, 'target anno filename:', tgt_anno_filename)
      # 이미지 파일의 경우 타겟 디렉토리로 단순 복사. 
      shutil.copy(src_img_filename, self.tgt_img_dir)
      # 타겟 annotation 출력 파일명으로 classid, bbox 좌표를 object 별로 생성. 
      with open(tgt_anno_filename, 'w', encoding='utf-8') as f:
        #print(k, v)
        # 여러개의 object 별로 classid와 bbox 좌표를 생성. 
        for obj in v:
          cat_name = self.coco_id_name_map.get(obj[1])
          # category_id는 class 명에 따라 0부터 순차적으로 부여. 
          category_id = self.coco_name_list.index(cat_name)
          #print('cat_name:', cat_name, 'category_id:', category_id)
          box = ['{:.6f}'.format(x) for x in obj[2]]
          box = ' '.join(box)
          line = str(category_id) + ' ' + box
          f.write(line + '\n')

  # ms-coco를 yolo format으로 변환. 
  def coco2yolo(self):
    print("loading image info...")
    images_info = self._load_images_info()
    print("loading done, total images", len(images_info))

    print("start converting...")
    anno_dict = self._convert_anno(images_info)
    print("converting done, total labels", len(anno_dict))

    print("saving txt file...")
    self._save_txt(anno_dict)
    print("saving done")

    //실행완료

In [ ]:
# 학습/검증/테스트용 images, labels 디렉토리 생성. 
!mkdir /content/gdrive/MyDrive/YOLOv5/pig;
!cd /content/gdrive/MyDrive/YOLOv5/pig; mkdir images; mkdir labels;
!cd /content/gdrive/MyDrive/YOLOv5/pig/images; mkdir train; mkdir val; mkdir test
!cd /content/gdrive/MyDrive/YOLOv5/pig/labels; mkdir train; mkdir val; mkdir test

//실행완료

In [ ]:
# train 용 yolo 데이터 세트 생성
train_yolo_converter = COCO2YOLO(src_img_dir='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/images', json_file='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/train.json',
                                 tgt_img_dir='/content/gdrive/MyDrive/YOLOv5/pig/images/train', tgt_anno_dir='/content/gdrive/MyDrive/YOLOv5/pig/labels/train')
train_yolo_converter.coco2yolo()

# val 용 yolo 데이터 세트 생성. 
val_yolo_converter = COCO2YOLO(src_img_dir='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/images', json_file='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/val.json',
                                 tgt_img_dir='/content/gdrive/MyDrive/YOLOv5/pig/images/val', tgt_anno_dir='/content/gdrive/MyDrive/YOLOv5/pig/labels/val')
val_yolo_converter.coco2yolo()

# test 용 yolo 데이터 세트 생성. 
test_yolo_converter = COCO2YOLO(src_img_dir='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/images', json_file='/content/gdrive/MyDrive/YOLOv5/data/Train.vol1/json/seed221112/test.json',
                                 tgt_img_dir='/content/gdrive/MyDrive/YOLOv5/pig/images/test', tgt_anno_dir='/content/gdrive/MyDrive/YOLOv5/pig/labels/test')
test_yolo_converter.coco2yolo()

//실행완료

total images 1350
total categories 1
total labels 34575
loading image info...
loading done, total images 1350
start converting...
converting done, total labels 1350
saving txt file...
saving done
total images 540
total categories 1
total labels 14941
loading image info...
loading done, total images 540
start converting...
converting done, total labels 540
saving txt file...
saving done
total images 810
total categories 1
total labels 21635
loading image info...
loading done, total images 810
start converting...
converting done, total labels 810
saving txt file...
saving done


In [ ]:
#!mv /content/pig/custom_data.yaml /content/gdrive/MyDrive/yolo/data/custom_data.yaml

mv: cannot stat '/content/pig/custom_data.yaml': No such file or directory


In [ ]:
##############여기서부터#################
# Google Drive 접근을 위한 Mount 적용. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# # soft link로 Google Drive Directory 연결. 
!ln -s /content/drive/My\ Drive/ /mydrive

!ls /mydrive
# Google Drive 밑에 Directory 생성. 이미 생성 되어 있을 시 오류 발생. 
!mkdir "/mydrive/ultra_workdir"

/mydrive
mkdir: cannot create directory ‘/mydrive/ultra_workdir’: No such file or directory


In [ ]:
import yaml
yaml.__version__

'6.0'

In [ ]:
 !cd /content/gdrive/MyDrive/YOLOv5/yolov5; python train.py --img 640 --batch 8 --epochs 50 --data /content/gdrive/MyDrive/YOLOv5/custom_data.yaml --weights yolov5l.pt \
                                     --project=/content/gdrive/MyDrive/YOLOv5/ultra_workdir --name pig --exist-ok 

train: weights=yolov5l.pt, cfg=, data=/content/gdrive/MyDrive/YOLOv5/custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/YOLOv5/ultra_workdir, name=pig, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
requirements: YOLOv5 requirements "ipython" "thop>=0.1.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.2 MB/s eta 0:00:00

requirements: 2 pac

In [ ]:
# image 파일 inference 
!cd /content/gdrive/MyDrive/YOLOv5/yolov5;python detect.py --source /content/gdrive/MyDrive/YOLOv5/pig/images/test/ \
                            --weights /content/gdrive/MyDrive/YOLOv5/ultra_workdir/pig/weights/best_s.pt --conf 0.2 \
                            --project=/content/gdrive/MyDrive/data/ --name=run_image --exist-ok --line-thickness 3

                            

detect: weights=['/content/gdrive/MyDrive/YOLOv5/ultra_workdir/pig/weights/best_s.pt'], source=/content/gdrive/MyDrive/YOLOv5/pig/images/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/gdrive/MyDrive/data/, name=run_image, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-239-gf33718f Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients
image 1/66 /content/gdrive/MyDrive/YOLOv5/pig/images/test/07d73716-fdb6-483b-a329-b5c202f0783c_vmj9_b4c3ad30-3e75-4d74-8adf-435dc261c11d_jpeg_jpg.rf.01c95271201dfeb948c54e8f02e1dd95.jpg: 640x640 (no detections), 11.6ms
image 2/66 /content/gdrive/MyDrive/YOLOv5/pig/images/te

In [ ]:
!cd /content/yolov5; python test.py --weights /mydrive/ultra_workdir/bccd/weights/best.pt  --data /content/gdrive/MyDrive/yolo/data/custom_data.yaml \
                           --project /content/data/output --name=test_result --exist-ok --img 640 --iou 0.65

                           //실행완료

python3: can't open file 'test.py': [Errno 2] No such file or directory
